# 基于Bi-GRU+CRF的快递单信息抽取

In [1]:
import paddle
import paddle.nn as nn

import paddlenlp
from paddlenlp.datasets import MapDataset
from paddlenlp.data import Stack, Tuple, Pad
from paddlenlp.layers import LinearChainCrf, ViterbiDecoder, LinearChainCrfLoss
from paddlenlp.metrics import ChunkEvaluator

D:\Anaconda3\envs\paddle_2.4.1\lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## 1. 数据准备

In [2]:
# 下载并解压数据集
from paddle.utils.download import get_path_from_url
URL = "https://paddlenlp.bj.bcebos.com/paddlenlp/datasets/waybill.tar.gz"
get_path_from_url(URL,"./")

for i, line in enumerate(open('data/train.txt')):
    if 0 < i < 5:
        print ('%d: ' % i, line.split()[0])
        print ('   ', line.split()[1])

1:  16620200077宣荣嗣甘肃省白银市会宁县河畔镇十字街金海超市西行50米
    T-BT-IT-IT-IT-IT-IT-IT-IT-IT-IT-IP-BP-IP-IA1-BA1-IA1-IA2-BA2-IA2-IA3-BA3-IA3-IA4-BA4-IA4-IA4-IA4-IA4-IA4-IA4-IA4-IA4-IA4-IA4-IA4-IA4-IA4-I
2:  13552664307姜骏炜云南省德宏傣族景颇族自治州盈江县平原镇蜜回路下段
    T-BT-IT-IT-IT-IT-IT-IT-IT-IT-IT-IP-BP-IP-IA1-BA1-IA1-IA2-BA2-IA2-IA2-IA2-IA2-IA2-IA2-IA2-IA2-IA3-BA3-IA3-IA4-BA4-IA4-IA4-IA4-IA4-IA4-IA4-I
3:  内蒙古自治区赤峰市阿鲁科尔沁旗汉林西街路南13701085390那峥
    A1-BA1-IA1-IA1-IA1-IA1-IA2-BA2-IA2-IA3-BA3-IA3-IA3-IA3-IA3-IA4-BA4-IA4-IA4-IA4-IA4-IT-BT-IT-IT-IT-IT-IT-IT-IT-IT-IT-IP-BP-I
4:  广东省梅州市大埔县茶阳镇胜利路13601328173张铱
    A1-BA1-IA1-IA2-BA2-IA2-IA3-BA3-IA3-IA4-BA4-IA4-IA4-IA4-IA4-IT-BT-IT-IT-IT-IT-IT-IT-IT-IT-IT-IP-BP-I


In [3]:
def convert_tokens_to_ids(tokens, vocab, oov_token=None):
    token_ids = []
    oov_id = vocab.get(oov_token) if oov_token else None
    for token in tokens:
        token_id = vocab.get(token, oov_id)
        token_ids.append(token_id)
    return token_ids


def load_dict(dict_path):
    vocab = {}
    i = 0
    for line in open(dict_path, 'r', encoding='utf-8'):
        key = line.strip('\n')
        vocab[key] = i
        i += 1
    return vocab


def load_dataset(datafiles):
    def read(data_path):
        with open(data_path, 'r', encoding='utf-8') as fp:
            next(fp)
            for line in fp.readlines():
                words, labels = line.strip('\n').split('\t')
                words = words.split('\002')
                labels = labels.split('\002')
                yield words, labels

    if isinstance(datafiles, str):
        return MapDataset(list(read(datafiles)))
    elif isinstance(datafiles, list) or isinstance(datafiles, tuple):
        return [MapDataset(list(read(datafile))) for datafile in datafiles]

train_ds, dev_ds, test_ds = load_dataset(datafiles=('data/train.txt', 'data/dev.txt', 'data/test.txt'))

label_vocab = load_dict('./data/tag.dic')
word_vocab = load_dict('./data/word.dic')

def convert_example(example):
    tokens, labels = example
    token_ids = convert_tokens_to_ids(tokens, word_vocab, 'OOV')
    label_ids = convert_tokens_to_ids(labels, label_vocab, 'O')
    return token_ids, len(token_ids), label_ids

train_ds.map(convert_example)
dev_ds.map(convert_example)
test_ds.map(convert_example)

In [4]:
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=word_vocab.get('OOV')),  # token_ids
    Stack(),  # seq_len
    Pad(axis=0, pad_val=label_vocab.get('O'))  # label_ids
): fn(samples)

train_loader = paddle.io.DataLoader(
    dataset=train_ds,
    batch_size=32,
    shuffle=True,
    drop_last=True,
    return_list=True,
    collate_fn=batchify_fn)

dev_loader = paddle.io.DataLoader(
    dataset=dev_ds,
    batch_size=32,
    drop_last=True,
    return_list=True,
    collate_fn=batchify_fn)

test_loader = paddle.io.DataLoader(
    dataset=test_ds,
    batch_size=32,
    drop_last=True,
    return_list=True,
    collate_fn=batchify_fn)

## 2. 网络构建

In [5]:
class BiGRUWithCRF(nn.Layer):
    def __init__(self,
                 emb_size,
                 hidden_size,
                 word_num,
                 label_num,
                 use_w2v_emb=False):
        super(BiGRUWithCRF, self).__init__()
        if use_w2v_emb:
            self.word_emb = TokenEmbedding(
                extended_vocab_path='./conf/word.dic', unknown_token='OOV')
        else:
            self.word_emb = nn.Embedding(word_num, emb_size)
        self.gru = nn.GRU(emb_size,
                          hidden_size,
                          num_layers=2,
                          direction='bidirectional')
        self.fc = nn.Linear(hidden_size * 2, label_num + 2)  # BOS EOS
        self.crf = LinearChainCrf(label_num)
        self.decoder = ViterbiDecoder(self.crf.transitions)

    def forward(self, x, lens):
        embs = self.word_emb(x)
        output, _ = self.gru(embs)
        output = self.fc(output)
        _, pred = self.decoder(output, lens)
        return output, lens, pred

# Define the model netword and its loss
network = BiGRUWithCRF(300, 300, len(word_vocab), len(label_vocab))
model = paddle.Model(network)

In [6]:
optimizer = paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters())
crf_loss = LinearChainCrfLoss(network.crf)
chunk_evaluator = ChunkEvaluator(label_list=label_vocab.keys(), suffix=True)
model.prepare(optimizer, crf_loss, chunk_evaluator)

## 3. 模型训练

In [7]:
model.fit(train_data=train_loader,
          eval_data=dev_loader,
          epochs=1,
          save_dir='./results',
          log_freq=1)

[2024-06-07 17:35:23,915] [ WARNING] - Compatibility Warning: The params of LinearChainCrfLoss.forward has been modified. The third param is `labels`, and the fourth is not necessary. Please update the usage.


The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/1


ValueError: (InvalidArgument) The type of data we are trying to retrieve does not match the type of data currently contained in the container.
  [Hint: Expected dtype() == paddle::experimental::CppTypeToDataType<T>::Type(), but received dtype():7 != paddle::experimental::CppTypeToDataType<T>::Type():9.] (at ..\paddle\phi\core\dense_tensor.cc:143)


## 4. 模型评估

In [ ]:
model.evaluate(eval_data=test_loader, log_freq=1)